# Script Flete

In [4]:
import xlwings as xw
import pandas as pd

In [5]:
path = r"C:\Users\nil.monreal\Desktop\EZ Air Shipments Report.xlsx"

try:
    flete_wb = xw.Book(path, update_links=False)  # Open workbook
except Exception as e:
    print(f"Error: {e}")


In [6]:
act_sheet = flete_wb.sheets['Activity'] 

In [7]:
last_row = act_sheet.range('A3').end('down').row

last_row

3281

In [8]:
act_sheet.range(f"A3:AU{last_row}").api.AutoFilter(Field=44, Criteria1="<>", Operator=1)

True

In [9]:
# Define your specific range
visible_range = act_sheet.range(f"AU:AU").api.SpecialCells(12)

In [10]:
values = act_sheet.range(f"AU3:AU{last_row}").value

# Create DataFrame (skip header if needed)
df = pd.DataFrame(values[1:], columns=['AU'])

In [11]:
df

,AU
0,CH173806
1,CH174522
2,CH174688
3,CH174777
4,CH174543
...,...
3273,CH165358
3274,CH165359
3275,CH165360
3276,CH165361


In [12]:
def clean_au_value(value):
    if isinstance(value, str):
        # Remove double CH instances
        value = value.replace('CHCH', 'CH')
        # Add CH if missing and keep only digits after
        if not value.startswith('CH'):
            return 'CH' + ''.join(filter(str.isdigit, value))
        return 'CH' + ''.join(filter(str.isdigit, value[2:]))
    elif isinstance(value, (int, float)):
        # Handle numeric values by converting to string and adding CH
        return 'CH' + str(int(value))
    return value

# Apply the cleaning function to your AU column
df['AU'] = df['AU'].apply(clean_au_value)

In [13]:
df

,AU
0,CH173806
1,CH174522
2,CH174688
3,CH174777
4,CH174543
...,...
3273,CH165358
3274,CH165359
3275,CH165360
3276,CH165361


In [14]:
# Display unique values to check the patterns
print("Unique values in the column:")
print(df['AU'].unique())

# Check if all values start with 'CH'
print("\nValues not starting with 'CH':")
print(df[~df['AU'].str.startswith('CH')])

# Check the length of each value (should be 8 if format is CHxxxxxx)
print("\nValues not 8 characters long:")
print(df[df['AU'].str.len() != 8])

# Display value counts to see frequency of each value
print("\nValue counts:")
print(df['AU'].value_counts().head())

# Check for any remaining double 'CHCH'
print("\nValues containing 'CHCH':")
print(df[df['AU'].str.contains('CHCH', na=False)])

Unique values in the column:
['CH173806' 'CH174522' 'CH174688' ... 'CH165360' 'CH165361' 'CH93727']

Values not starting with 'CH':
Empty DataFrame
Columns: [AU]
Index: []

Values not 8 characters long:
                  AU
7            CH17492
48         CH1744444
73         CH1784453
84    CH174252174253
136          CH74136
...              ...
3230         CH93762
3235         CH93781
3237         CH93791
3239         CH93761
3277         CH93727

[165 rows x 1 columns]

Value counts:
AU
CH          54
CH169672     2
CH16986      2
CH171312     2
CH171310     2
Name: count, dtype: int64

Values containing 'CHCH':
Empty DataFrame
Columns: [AU]
Index: []


In [15]:
# Convert DataFrame column to list of values
cleaned_values = df['AU'].tolist()

for i, value in enumerate(cleaned_values, start=4):
    act_sheet.range(f'AU{i}').value = value

In [16]:
flete_wb.save()
flete_wb.close()